In [3]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from helita.sim import rh15d 
from helita.vis import rh15d_vis

from scipy.integrate import quadrature
from scipy.integrate import cumtrapz

from pathlib import Path

dpath = Path("/Users/elka127/Documents/code/rh/rh15d/run/output")

In [59]:
data = rh15d.Rh15dout(dpath)
wave = data.ray.wavelength

OSError: [Errno group not found: atom_H] 'atom_H'

In [53]:
indices = np.arange(len(wave))[(wave > 279 ) & (wave < 280.5)]

In [54]:
plt.plot(wave[indices],data.ray.intensity[0,0,indices])
plt.title(r'Mg II h&k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Mg II h&k')

In [55]:
#Just to show how many wavelength points 
print(data.ray.intensity[0,0,indices].shape)

(418,)


### Separate run with a customa made wavetable. 

In [43]:
dpath_wavetable ='/Users/elka127/Documents/code/rh/rh15d/run/output'

In [44]:
data_wavetable = rh15d.Rh15dout(dpath_wavetable)
wave1 = data_wavetable.ray.wavelength

--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_aux.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_indata.hdf5 file.
--- Read /Users/elka127/Documents/code/rh/rh15d/run/output/output_ray.hdf5 file.


In [45]:
indices1 = np.arange(len(wave1))[(wave1 > 279 ) & (wave1 < 280.5)]

In [46]:
plt.plot(wave1[indices1],data_wavetable.ray.intensity[0,0,indices1])
plt.title(r'Mg II h&k')

Text(0.5, 1.0, 'Mg II h&k')

In [51]:
## Run with this wavetable first. 
IRIS_wave = rh15d.read_wave_file('/Users/elka127/Documents/code/rh/Atoms/wave_files/IRIS_NUV_full.wave')## Only use filename.wave. RH is specific about this.

In [52]:
## Check how many wavelength points are there in this wavelength file. 
## You can compare the time needed with different wavetable or no wavetable. Computation time scales with that. 
## in the run_example folder (or run folder) you will have a file called time.out It gives the computation time in seconds. 
IRIS_wave.shape

(1792,)

## Custom made wavelength array for RH1.5D

In [53]:
IRIS_wave[0],IRIS_wave[-1]

(278.25489792475247, 283.3965278911155)

In [55]:
## Run RH with this wavetable again. 
rh15d.make_wave_file('/Users/elka127/Documents/code/rh/Atoms/wave_files/IRIS_NUV_full.wave',IRIS_wave[0] ,IRIS_wave[-1], 0.001)

Wrote 5142 wavelengths to file.


### Make sure to save the output files in a separate folder. It gets overwritten otherwise

In [ ]:
dpath_linelist = '/mn/stornext/u3/souvikb/rh/rh15d/run_example/output_linelist/' ## Like this
data_linelist= rh15d.Rh15dout(dpath_linelist)

In [ ]:
wave2 = data_linelist.ray.wavelength
indices2 = np.arange(len(wave2))[(wave2 > 279 ) & (wave2 < 280.5)]

In [ ]:
plt.plot(wave2[indices2],data_linelist.ray.intensity[0,0,indices2])
plt.title(r'Mg II h&k with linelist')

In [ ]:
plt.plot(wave2[indices2],data_linelist.ray.tau_one_height[0,0,indices2]/1e6,color='red')
plt.xlabel('$\lambda$ (nm)')
plt.ylabel('H [Mm]')

# With the Bifrost Snapshot

In [4]:
import h5py
from mpl_toolkits.axes_grid1 import make_axes_locatable
dpath_sim = "/Users/elka127/Documents/uio/project-Mflare/mflares/"

In [5]:
# Alternate way of reading the hdf5 files
hf = h5py.File(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf","r")
#hf = h5py.File(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf","r")


In [4]:
# To see what data is in this file, we can call the keys() method on the file object.
hf.keys() # variables in the Bifrost snapshot

<KeysViewHDF5 ['electron_density', 'hydrogen_populations', 'nhydr', 'nt', 'nx', 'ny', 'nz', 'snapshot_number', 'temperature', 'velocity_z', 'x', 'y', 'z']>

In [68]:
# Display some variables 
temp = hf['temperature']
vlos = hf['velocity_z'] # LOS_V
eden = hf['electron_density']
#Blos = hf['B_z'] # LOS B
H = hf['z'][0,:] # geometric height
snapshot_number = hf["snapshot_number"]
nhydr = hf["nhydr"]
x = hf["x"]
z = hf["z"]

(52, 1, 249)


In [47]:
#print(H[:]/1e6)
#print("---")
#print(np.asarray(snapshot_number))


print(np.asarray(x))
print("---")
print(snapshot_number)
#print("---")

#print(np.asarray(temp))

[10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27.
 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40. 41. 42. 43. 44. 45.
 46. 47. 48. 49. 50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61.]
---
<HDF5 dataset "snapshot_number": shape (1,), type "<i4">


In [6]:
import xarray
atmos = xarray.open_dataset(dpath_sim+"radyn_out.F13_dpl_GRID_pt1s.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m2F12-37.max2F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.m5F12-37.max5F12_Ec37_d3.ncdf")
#atmos = xarray.open_dataset(dpath_sim+"tx.mF13-37.maxF13_Ec37_d3_dt0.1_10s.ncdf")
atmos

<xarray.Dataset>
Dimensions:               (nhydr: 6, nt: 1, nx: 52, ny: 1, nz: 249)
Dimensions without coordinates: nhydr, nt, nx, ny, nz
Data variables:
    temperature           (nt, nx, ny, nz) float32 ...
    velocity_z            (nt, nx, ny, nz) float32 ...
    electron_density      (nt, nx, ny, nz) float64 ...
    hydrogen_populations  (nt, nhydr, nx, ny, nz) float32 ...
    snapshot_number       (nt) int32 ...
    x                     (nx) float32 ...
    y                     (ny) float32 ...
    z                     (nt, nx, ny, nz) float64 ...
Attributes:
    description:     2.5324933e-71
    has_B:        0

In [7]:
#atmos.temperature.plot.hist()
fig, ax = plt.subplots()
#atmos.temperature.plot()
#atmos.velocity_z.plot()

atmos.electron_density.plot()
ax.set_xlabel('height')
ax.set_ylabel('time')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'time')

In [ ]:
# Display their shape. Always try to make sense of what you read.
temp.shape, vlos.shape, Blos.shape ## format is (snapshot, nx, ny, nz)

In [61]:
#fig, ax = plt.subplots()
#atmos.temperature.plot()
#atmos.velocity_z.plot()
#atmos.electron_density.plot()
#temperature = hf['temperature']
#print(temperature.shape)
#plt.plot(
#    atmos.electron_density[0,22,0]
#)
#plt.plot(
#    atmos.velocity_z[0,22,0]
#)
plt.plot(
    atmos.temperature[0,22,0]
)
ax.set_xlabel('height')
ax.set_ylabel('time')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(111.69444444444443, 0.5, 'time')

In [14]:
plt.figure()
for i in range(1):
    plt.plot(
        data.atmos.height_scale[0,0]/1e6, 
        data.atmos.electron_density[i, 22, 0],
        label='Level %i' % (i + 1)
    )
plt.legend(loc="upper left")
plt.xlabel("Height (Mm)")
plt.ylabel("Departure coefficients")
plt.xlim(0,11)
plt.ylim(-20,10000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'data' is not defined

### You can also display the constituents of any .hdf5 file by using the command: ncdump -h filename.hdf5 in the terminal

In [48]:
# Suppose you want to display the variables at a particular height. But you may not know the exact height in decimals. So use the following
def find_nearest(array, value):
    '''This method finds the nearest value 
    in a given array.'''
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx # this is the index corresponding to value you provde

In [49]:
print(find_nearest(np.array(H/1e6),0.3)) #Index corresponding to the nearest value 0.3 Mm

707


## Sample visualization of atmospheric params from the Bifrost atmosphere

In [66]:
fig, axs =plt.subplots(1,3,figsize=(15,5),facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = 0.1,wspace=0.25,left=0.05,right=0.95,top=0.93,bottom=0.07)
axs.ravel()
im=axs[0].imshow(temp[0,:,:,248],cmap='plasma')
divider1 = make_axes_locatable(axs[0])
cax0 = divider1.append_axes("right", size="5%", pad=0.05)
cbar=plt.colorbar(im,cax=cax0)
cbar.set_label('T (K)')
axs[0].set_xlabel('pixels')
axs[0].set_ylabel('pixels')

im=axs[1].imshow(vlos[0,:,:,248]/1e3,cmap='bwr',vmax=1.5,vmin=-1.5)
divider1 = make_axes_locatable(axs[1])
cax0 = divider1.append_axes("right", size="5%", pad=0.05)
cbar=plt.colorbar(im,cax=cax0)
cbar.set_label('V (km/s)')
#axs[1].set_xlabel('pixels')
#axs[1].set_ylabel('pixels')

#im=axs[2].imshow(Blos[0,:,:,248]*1e4,cmap='BrBG',vmax=20,vmin=-100)
#divider1 = make_axes_locatable(axs[2])
#cax0 = divider1.append_axes("right", size="5%", pad=0.05)
#cbar=plt.colorbar(im,cax=cax0)
#cbar.set_label('B (G)')
#axs[2].set_xlabel('pixels')
#axs[2].set_ylabel('pixels')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Your task would to use this atmosphere as input for RH1.5D and synthesie Ca II 854.2 nm line 